In [ ]:
import funcs.data_wrangling as dw
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib.pyplot as plt
#import plotly.express as px
#from matplotlib import rcParams
#from statsmodels.tsa.seasonal import STL
#from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsforecast import StatsForecast
from statsforecast.models import (
    AutoARIMA,
    HoltWinters,
    CrostonClassic as Croston, 
    HistoricAverage,
    DynamicOptimizedTheta as DOT,
    SeasonalNaive
)
#from datasetsforecast.losses import mse, mae, rmse
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    r2_score
)
from sktime.performance_metrics.forecasting import     MeanAbsolutePercentageError
#rcParams['figure.figsize'] = 15, 5

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = dw.ons_data(freq='h', ano_inicio=2000, ano_fim=2023, idreg="S")
df = dw.pipeline(data, update=False)

In [3]:
df.head()

,id_reg,desc_reg,load_mwmed
date,,,
2000-01-01 00:00:00,S,SUL,5777.0
2000-01-01 01:00:00,S,SUL,5580.7
2000-01-01 02:00:00,S,SUL,5098.7
2000-01-01 03:00:00,S,SUL,4753.7
2000-01-01 04:00:00,S,SUL,4584.1


In [4]:
# Último ano
df_ly = df.iloc[-(24*365):,:]
df_ly.head()

,id_reg,desc_reg,load_mwmed
date,,,
2022-03-07 00:00:00,S,SUL,12638.106
2022-03-07 01:00:00,S,SUL,12158.367
2022-03-07 02:00:00,S,SUL,11829.226
2022-03-07 03:00:00,S,SUL,11627.201
2022-03-07 04:00:00,S,SUL,11574.747


In [5]:
df2 = dw.prepare_statsforecast_df(df_ly, "hourly_load")

In [6]:
df2.shape

(8760, 3)

# AutoArima

In [7]:
sf = StatsForecast(
    models= [AutoARIMA(season_length=24)],
    freq='H'
)

In [8]:
sf.fit(df2)

# Multiple models

In [9]:
models = [
    AutoARIMA(season_length=24),
    HoltWinters(),
    Croston(),
    SeasonalNaive(season_length=24),
    HistoricAverage(),
    DOT(season_length=24)
]

In [52]:
sf = StatsForecast(
    df=df2, 
    models=models,
    freq='H', 
    n_jobs=-1,
    fallback_model = SeasonalNaive(season_length=24)
)

In [53]:
forecasts_df = sf.forecast(h=48, level=[90])

forecasts_df.head()

,ds,AutoARIMA,AutoARIMA-lo-90,AutoARIMA-hi-90,HoltWinters,HoltWinters-lo-90,HoltWinters-hi-90,CrostonClassic,SeasonalNaive,SeasonalNaive-lo-90,SeasonalNaive-hi-90,HistoricAverage,HistoricAverage-lo-90,HistoricAverage-hi-90,DynamicOptimizedTheta,DynamicOptimizedTheta-lo-90,DynamicOptimizedTheta-hi-90
unique_id,,,,,,,,,,,,,,,,,
hourly_load,2023-03-07 00:00:00,12242.286133,11727.636719,12756.936523,11205.140625,7196.831055,15213.450195,13723.993164,11205.140625,7196.831055,15213.450195,11627.321289,7858.384766,15396.257812,12119.828125,11590.079102,12788.010742
hourly_load,2023-03-07 01:00:00,11503.185547,10626.280273,12380.089844,10703.818359,6695.508789,14712.127930,13723.993164,10703.818359,6695.508789,14712.127930,11627.321289,7858.384766,15396.257812,11409.166992,10655.356445,12196.412109
hourly_load,2023-03-07 02:00:00,11013.065430,9881.473633,12144.656250,10415.939453,6407.629883,14424.249023,13723.993164,10415.939453,6407.629883,14424.249023,11627.321289,7858.384766,15396.257812,10958.120117,9908.471680,11853.031250
hourly_load,2023-03-07 03:00:00,10766.224609,9474.901367,12057.546875,10295.325195,6287.015625,14303.634766,13723.993164,10295.325195,6287.015625,14303.634766,11627.321289,7858.384766,15396.257812,10772.516602,9728.506836,11670.973633
hourly_load,2023-03-07 04:00:00,10701.494141,9319.984375,12083.002930,10342.305664,6333.996094,14350.615234,13723.993164,10342.305664,6333.996094,14350.615234,11627.321289,7858.384766,15396.257812,10793.220703,9716.607422,11882.041016


In [54]:
sf.plot(df2,forecasts_df)

In [55]:
crossvaldation_df = sf.cross_validation(
    df=df2,
    h=24,
    step_size=24,
    n_windows=2
  )

crossvaldation_df.head()

,ds,cutoff,y,AutoARIMA,HoltWinters,CrostonClassic,SeasonalNaive,HistoricAverage,DynamicOptimizedTheta
unique_id,,,,,,,,,
hourly_load,2023-03-05 00:00:00,2023-03-04 23:00:00,10916.413086,11284.436523,12685.149414,12377.345703,12685.149414,11627.598633,10690.840820
hourly_load,2023-03-05 01:00:00,2023-03-04 23:00:00,10333.477539,11007.217773,11982.125000,12377.345703,11982.125000,11627.598633,10063.570312
hourly_load,2023-03-05 02:00:00,2023-03-04 23:00:00,9867.043945,10882.205078,11456.279297,12377.345703,11456.279297,11627.598633,9665.756836
hourly_load,2023-03-05 03:00:00,2023-03-04 23:00:00,9549.212891,10914.300781,11201.987305,12377.345703,11201.987305,11627.598633,9502.586914
hourly_load,2023-03-05 04:00:00,2023-03-04 23:00:00,9347.792969,11006.535156,11050.782227,12377.345703,11050.782227,11627.598633,9521.680664


In [61]:
def evaluate_cross_validation(df, metric):
    models = df.drop(columns=['ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    for model in models:
        eval_ = df.groupby(['unique_id', 'cutoff']).apply(lambda x: metric(x['y'].values, x[model].values)).to_frame() # Calculate loss for every unique_id, model and cutoff.
        eval_.columns = [model]
        evals.append(eval_)
    evals = pd.concat(evals, axis=1)
    evals = evals.groupby(['unique_id']).mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals

In [63]:
evaluation_df = evaluate_cross_validation(crossvaldation_df, mean_squared_error)

evaluation_df.head()

,AutoARIMA,HoltWinters,CrostonClassic,SeasonalNaive,HistoricAverage,DynamicOptimizedTheta,best_model
unique_id,,,,,,,
hourly_load,6284109.5,8966975.0,8028957.0,8966975.0,5386649.0,3352157.5,DynamicOptimizedTheta


In [64]:
metrics = [
    mean_absolute_error,
    mean_squared_error,
    mean_squared_error,
    mean_absolute_percentage_error,
    r2_score,
    MeanAbsolutePercentageError(symmetric=True)
]

In [65]:
metrics_df = pd.DataFrame()
for metric in metrics:
    evaluation_df = evaluate_cross_validation(crossvaldation_df, metric)
    try:
        evaluation_df["metric"] = metric.__name__
    except:
        evaluation_df["metric"] = type(metric).__name__
    metrics_df = pd.concat([metrics_df, evaluation_df])
metrics_df

,AutoARIMA,HoltWinters,CrostonClassic,SeasonalNaive,HistoricAverage,DynamicOptimizedTheta,best_model,metric
unique_id,,,,,,,,
hourly_load,2.126906e+03,2.487894e+03,2.468536e+03,2.487894e+03,2.077946e+03,1.477993e+03,DynamicOptimizedTheta,mean_absolute_error
hourly_load,6.284110e+06,8.966975e+06,8.028957e+06,8.966975e+06,5.386649e+06,3.352158e+06,DynamicOptimizedTheta,mean_squared_error
hourly_load,6.284110e+06,8.966975e+06,8.028957e+06,8.966975e+06,5.386649e+06,3.352158e+06,DynamicOptimizedTheta,mean_squared_error
hourly_load,1.993308e-01,2.170365e-01,2.281773e-01,2.170365e-01,1.909619e-01,1.421755e-01,DynamicOptimizedTheta,mean_absolute_percentage_error
hourly_load,-1.125702e+00,-2.154977e+00,-1.729837e+00,-2.154977e+00,-8.338373e-01,-1.168084e-01,HoltWinters,r2_score
hourly_load,1.869442e-01,2.268995e-01,2.146225e-01,2.268995e-01,1.819776e-01,1.314299e-01,DynamicOptimizedTheta,MeanAbsolutePercentageError


In [66]:
summary_df = evaluation_df.groupby('best_model').size().sort_values().to_frame()

summary_df.reset_index().columns = ["Model", "Nr. of unique_ids"]

summary_df

,0
best_model,
DynamicOptimizedTheta,1
